user_df 만들기

In [36]:
# user_df: 중복되는 이름 없이 아이디, 이름, 나이 데이터가 포함된 데이터 프레임 생성
user_df = pd.DataFrame(columns=["UserID", "Name", "Age"])

for idx in range(1,9):    # 무한루프에 빠지지 않도록 위에서 넣어준 이름보다 적게 넣기
    name = get_name()

    # 이름이 중복이면 다시 뽑기
    while name in list(user_df["Name"]):
        name = get_name()
    # 데이터 name_df에 insert
    data = {"Name":name, "UserID":idx, "Age":get_age()}
    user_df.loc[len(user_df)] = data    # 마지막 줄에 데이터 추가

    
user_df

,UserID,Name,Age
0,1,Baldy,32
1,2,Alan,34
2,3,Adam,27
3,4,Andrew,36
4,5,Peter,22
5,6,Arnold,36
6,7,Jim,28
7,8,Alvin,24


money_df 만들기

In [37]:
# money_df: ID와 Money 데이터가 있는 데이터 프레임 생성
money_df = pd.DataFrame(columns=["ID","Money"])

for idx in range(15):
    money = random.randint(1,20) * 1000
    data = {"Money":money, "ID":random.randint(1,9)}
    money_df.loc[len(money_df)] = data


# money_df.sort_values("ID", inplace=True)
money_df

,ID,Money
0,6,16000
1,9,3000
2,7,3000
3,2,6000
4,7,11000
5,6,4000
6,9,18000
7,9,15000
8,5,10000
9,6,6000


money_df의 ID 컬럼과 user_df의 UserID(key값)를 매칭 시켜 merge

In [38]:
# UserID와 ID 데이터는 같지만 컬럼명이 다르기 때문에 두개의 컬럼 모두 데이터 프레임에 들어감
result_df = money_df.merge(user_df, left_on="ID", right_on="UserID", how='outer')
result_df

,ID,Money,UserID,Name,Age
0,6,16000,6,Arnold,36
1,6,4000,6,Arnold,36
2,6,6000,6,Arnold,36
3,9,3000,NaN,NaN,NaN
4,9,18000,NaN,NaN,NaN
5,9,15000,NaN,NaN,NaN
6,7,3000,7,Jim,28
7,7,11000,7,Jim,28
8,7,20000,7,Jim,28
9,2,6000,2,Alan,34


In [39]:
# 컬럼명을 동일하게 변경한후에 merge
# left_on과 right_on을 적어줄 필요가 없다.
user_df.rename(columns={"UserID":"ID"}, inplace=True)
result_df = pd.merge(money_df, user_df)
result_df

,ID,Money,Name,Age
0,6,16000,Arnold,36
1,6,4000,Arnold,36
2,6,6000,Arnold,36
3,7,3000,Jim,28
4,7,11000,Jim,28
5,7,20000,Jim,28
6,2,6000,Alan,34
7,2,10000,Alan,34
8,5,10000,Peter,22
9,4,12000,Andrew,36


In [40]:
# 각 이름으로 groubpy하고 Money 데이터를 모두 sum한 결과의 인덱스를 리셋한다.
money_list = result_df.groupby("Name").sum()["Money"].reset_index()

# Money데이터를 내림차순으로 정렬하고 index를 리셋한다.
money_list = money_list.sort_values(by=['Money'], ascending=False).reset_index(drop=True)
money_list

,Name,Money
0,Jim,34000
1,Arnold,26000
2,Alan,16000
3,Alvin,14000
4,Andrew,12000
5,Peter,10000
6,Baldy,5000


In [42]:
# how에 outer를 사용하면 데이터가 없는 사람은 Money가 NaN으로 출력된다.
# fillna(value=0) : NaN 데이터를 0으로 채운다.
result = pd.merge(user_df, money_list, how='outer').fillna(value=0)
result = result.sort_values(by=['Money'], ascending=False).reset_index(drop=True)
result

,ID,Name,Age,Money
0,7,Jim,28,34000.0
1,6,Arnold,36,26000.0
2,2,Alan,34,16000.0
3,8,Alvin,24,14000.0
4,4,Andrew,36,12000.0
5,5,Peter,22,10000.0
6,1,Baldy,32,5000.0
7,3,Adam,27,0.0


In [45]:
# column의 형변환
result["Money"] = result["Money"].astype("int")
result

,ID,Name,Age,Money
0,7,Jim,28,34000
1,6,Arnold,36,26000
2,2,Alan,34,16000
3,8,Alvin,24,14000
4,4,Andrew,36,12000
5,5,Peter,22,10000
6,1,Baldy,32,5000
7,3,Adam,27,0
